In [3]:
!pip -q install huggingface_hub langchain langchain_community sentence-transformers huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.7 MB/s eta 0:00:00


In [66]:
import torch
from langchain import PromptTemplate, LLMChain
from huggingface_hub import HfApi
import os
from langchain.llms import HuggingFaceHub
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "..."
import re

In [67]:
class pipeline():
  def __init__(self, model, temperature, max_new_tokens, max_length):
    self.model = model
    self.temperature = temperature
    self.max_new_tokens = max_new_tokens
    self.max_length = max_length

  def predict(self, prompt):
    llm = HuggingFaceHub(
    repo_id=self.model,
    model_kwargs={
        "temperature": self.temperature,
        "max_length": self.max_length,
        "max_new_tokens": self.max_new_tokens,
    }
    )
    answer = llm(prompt)
    return answer

In [68]:
prompt_template = """
You are a helpful AI assistant who provides 2 queries that mean essentially the same as the asked query.
Do not try to answer the query.
Do the aforementioned task for the given query:
{question}
"""

In [69]:
class assistant_model(pipeline):
  def __init__(self, model, prompt_template, query, temperature, max_new_tokens, max_length):
    super().__init__(model, temperature, max_new_tokens, max_length)
    self.prompt_template = prompt_template
    self.query = query

  def QA_decomposition(self):
      formatted_query = self.prompt_template.format(question=self.query)
      raw_answer = self.predict(formatted_query)
      match = re.search(r"(1\..+)\n(2\..+)", raw_answer, re.DOTALL)
      if match:
          query1 = match.group(1).strip()
          query2 = match.group(2).strip()
          return f"{self.query}\n{query1}\n{query2}"
      else:
          return raw_answer.strip()

In [77]:
model = "meta-llama/Llama-3.2-3B-Instruct"
query = "Who won the football world cup 2024?"
temperature = 0.7
max_new_tokens = 500
max_length = 200

assistant = assistant_model(model, prompt_template, query, temperature, max_new_tokens, max_length)

result = assistant.QA_decomposition()
print(result)


Who won the football world cup 2024?
1. What was the winner of the FIFA World Cup competition in the year 2024?
2. Who is the champion of the FIFA World Cup that took place in the year 2024?
